In [1]:
import pandas as pd
from scipy.stats import zscore

In [2]:
product_data="../data/product_data.csv"
attribute_data="../data/attribute_data.csv"

product_df=pd.read_csv(product_data)
attribute_df=pd.read_csv(attribute_data)

In [3]:
product_df = product_df.drop_duplicates()

In [4]:
product_df=product_df.dropna(subset=['des_color'])

In [5]:
unique_values=sorted(product_df['cod_color'].unique())
print(unique_values)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', 'BB', 'BL', 'CG', 'CO', 'CU', 'DC', 'DI', 'DO', 'GC', 'GM', 'OR', 'PL', 'TA', 'TC', 'TD', 'TG', 'TL', 'TM', 'TN', 'TO', 'TS', 'TU']


In [6]:
string_values = [value for value in unique_values if not value.isdigit()]
mapping = {value: 100 + idx for idx, value in enumerate(string_values)}
product_df['cod_color'] = product_df['cod_color'].replace(mapping)

In [7]:
product_df['cod_color']=product_df['cod_color'].astype(int)
product_df['cod_color'] = zscore(product_df['cod_color'])

In [8]:
def one_hot_encoding(column):
    global product_df
    one_hot_encoded_column= pd.get_dummies(product_df[column]) # Perform one-hot encoding
    product_df = pd.concat([product_df, one_hot_encoded_column], axis=1) # Concatenate the one-hot encoded columns with the original dataframe
    product_df.drop(column, axis=1, inplace=True) # Remove the original column 
    return product_df

one_hot_encoding('des_sex')
one_hot_encoding('des_age')
one_hot_encoding('des_line')
one_hot_encoding('des_fabric')
one_hot_encoding('des_product_category')
one_hot_encoding('des_product_aggregated_family')
one_hot_encoding('des_product_family')
one_hot_encoding('des_product_type')
one_hot_encoding('des_color')


,cod_modelo_color,cod_color,des_filename,Female,Male,Unisex,Adult,Baby,Kids,Newborn,...,TESTA DI MORO,TINTA,TOPO,TURQUESA,VAINILLA,VERDE,VERDE PASTEL,VINO,VIOLETA,VISON
0,83_1124642,0.704180,83_1124642_17074019-82_B.jpg,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,86_1215223,-1.357636,86_1215223_47014392-01_B.jpg,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,84_1167695,0.398726,84_1167695_27005150-70_B.jpg,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,82_1108473,-1.357636,82_1108473_87076320-01_B.jpg,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,83_1137778,-0.441273,83_1137778_17014765-37_.jpg,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61479,84_1171997,-1.077636,84_1171997_27004019-12_B.jpg,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61480,84_1179998,-0.110365,84_1179998_27028257-50_B.jpg,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
61481,83_1131136,-0.059456,83_1131136_17062885-52_B.jpg,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
61482,82_1103532,0.042363,82_1103532_87075684-56_B.jpg,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
attribute_df=attribute_df.drop('cod_value', axis=1)

In [10]:
des_value_list = []
attribute_indices_dict = {}
des_value_to_index = {}

current_index = 0

for attribute_name, group in attribute_df.groupby("attribute_name"):
    # Get unique des_value within this attribute
    unique_values = group["des_value"].unique().tolist()
    
    # Record start and end indices for this attribute
    start_index = current_index
    end_index = current_index + len(unique_values) - 1
    attribute_indices_dict[attribute_name] = (start_index, end_index)
    
    # Add des_values to des_value_list and create mapping to indices
    for i, des_value in enumerate(unique_values):
        index_in_list = current_index + i
        des_value_list.append(des_value)
        des_value_to_index[(attribute_name, des_value)] = index_in_list
    
    current_index = end_index + 1

# Total number of elements in des_value_list
total_des_values = len(des_value_list)

# Step 2: Create label lists for each cod_modelo_color
# Get the list of unique 'cod_modelo_color' values
unique_cod_modelo_colors = attribute_df['cod_modelo_color'].unique()

# Create a dictionary to hold the label lists for each 'cod_modelo_color'
cod_modelo_color_to_label = {}

for cod_modelo_color in unique_cod_modelo_colors:
    # Get all rows with this cod_modelo_color
    subset = attribute_df[attribute_df['cod_modelo_color'] == cod_modelo_color]
    
    # Get the set of (attribute_name, des_value) pairs
    attribute_des_value_pairs = subset[['attribute_name', 'des_value']].drop_duplicates().values.tolist()
    
    # Initialize label list with zeros
    label_list = [0] * total_des_values
    
    # Set positions corresponding to des_values to 1
    for attribute_name, des_value in attribute_des_value_pairs:
        index = des_value_to_index.get((attribute_name, des_value))
        if index is not None:
            label_list[index] = 1
        else:
            print(f"Warning: ({attribute_name}, {des_value}) not found in des_value_list.")
    
    # Assign the label_list to the cod_modelo_color
    cod_modelo_color_to_label[cod_modelo_color] = label_list

# Step 3: Assign the label lists back to the DataFrame
# Create a DataFrame from the cod_modelo_color_to_label dictionary
df_labels = pd.DataFrame({
    'cod_modelo_color': list(cod_modelo_color_to_label.keys()),
    'label': list(cod_modelo_color_to_label.values())
})



# Now, df_attribute_data contains the 'label' column with the list of labels
print(attribute_df)

       cod_modelo_color     attribute_name         des_value
0            85_1202950    silhouette_type              Slim
1            86_1217677    silhouette_type          Oversize
2            84_1168477    silhouette_type              Slim
3            82_1101897    silhouette_type              Slim
4            84_1162605    silhouette_type             Recto
...                 ...                ...               ...
151107       85_1209636  closure_placement  Cierre Delantero
151108       85_1180638  closure_placement  Cierre Delantero
151109       86_1226349  closure_placement  Cierre Delantero
151110       86_1213782  closure_placement  Cierre Delantero
151111       84_1160746  closure_placement        Sin cierre

[151112 rows x 3 columns]


In [11]:
"""attribute=[]
value=[]

grouped = attribute_df.groupby('attribute_name')[['attribute_name', 'des_value']].apply(lambda x: x.drop_duplicates()).reset_index(drop=True)
for name, group in grouped.groupby('attribute_name'):
    for idx, row in group.iterrows():
        attribute.append(row[0])
        value.append(row[1])

ids=attribute_df['cod_modelo_color'].unique()

label_column=[]

for id in ids:
    label_id=[]
    for atr, val in zip(attribute, value):
        df1 = pd.DataFrame([[id, atr, val]], columns=['cod_modelo_color', 'attribute_name', 'des_value'])
        row_exists = df1.apply(lambda row: (attribute_df == row).all(axis=1).any(), axis=1)
        label_id.append(row_exists)
    label_column.append(label_id)"""

"attribute=[]\nvalue=[]\n\ngrouped = attribute_df.groupby('attribute_name')[['attribute_name', 'des_value']].apply(lambda x: x.drop_duplicates()).reset_index(drop=True)\nfor name, group in grouped.groupby('attribute_name'):\n    for idx, row in group.iterrows():\n        attribute.append(row[0])\n        value.append(row[1])\n\nids=attribute_df['cod_modelo_color'].unique()\n\nlabel_column=[]\n\nfor id in ids:\n    label_id=[]\n    for atr, val in zip(attribute, value):\n        df1 = pd.DataFrame([[id, atr, val]], columns=['cod_modelo_color', 'attribute_name', 'des_value'])\n        row_exists = df1.apply(lambda row: (attribute_df == row).all(axis=1).any(), axis=1)\n        label_id.append(row_exists)\n    label_column.append(label_id)"

In [12]:
attribute_df=attribute_df.drop('des_value', axis=1)

In [13]:
def one_hot_encoding(column):
    global attribute_df
    one_hot_encoded_column= pd.get_dummies(attribute_df[column]) # Perform one-hot encoding
    attribute_df = pd.concat([attribute_df, one_hot_encoded_column], axis=1) # Concatenate the one-hot encoded columns with the original dataframe
    attribute_df.drop(column, axis=1, inplace=True)

one_hot_encoding('attribute_name')

In [14]:
attribute_df = attribute_df.groupby('cod_modelo_color')['cane_height_type', 'closure_placement', 'heel_shape_type', 'knit_structure', 'length_type', 'neck_lapel_type', 'silhouette_type', 'sleeve_length_type', 'toecap_type', 'waist_type', 'woven_structure'].sum().reset_index()
attribute_df

C:\Users\arroc\AppData\Local\Temp\ipykernel_6176\1521431674.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  attribute_df = attribute_df.groupby('cod_modelo_color')['cane_height_type', 'closure_placement', 'heel_shape_type', 'knit_structure', 'length_type', 'neck_lapel_type', 'silhouette_type', 'sleeve_length_type', 'toecap_type', 'waist_type', 'woven_structure'].sum().reset_index()


,cod_modelo_color,cane_height_type,closure_placement,heel_shape_type,knit_structure,length_type,neck_lapel_type,silhouette_type,sleeve_length_type,toecap_type,waist_type,woven_structure
0,81_1034451,0,0,0,0,0,1,1,1,0,0,0
1,81_1034525,0,0,0,0,0,1,1,1,0,0,0
2,81_1035318,0,0,0,0,1,0,0,0,0,0,0
3,81_1035321,0,0,0,0,1,0,0,0,0,0,0
4,81_1035361,0,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
33438,86_9893076,0,0,0,0,1,1,1,1,0,0,1
33439,86_9893077,0,0,0,0,1,1,1,1,0,0,1
33440,86_9893868,0,0,0,0,1,1,1,1,0,0,1
33441,86_9893898,0,1,0,0,1,1,1,0,0,0,1


In [15]:
# Merge the labels back to the original DataFrame
attribute_df = attribute_df.merge(df_labels, on='cod_modelo_color', how='left')

In [16]:
attribute_df['cod_modelo_color'].nunique()

33443

In [17]:
product_df['cod_modelo_color'].nunique()

33376

In [18]:
result=pd.merge(product_df, attribute_df, on='cod_modelo_color', how='inner')
result

,cod_modelo_color,cod_color,des_filename,Female,Male,Unisex,Adult,Baby,Kids,Newborn,...,heel_shape_type,knit_structure,length_type,neck_lapel_type,silhouette_type,sleeve_length_type,toecap_type,waist_type,woven_structure,label
0,83_1124642,0.704180,83_1124642_17074019-82_B.jpg,1,0,0,0,0,1,0,...,0,1,1,1,1,1,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,83_1124642,0.704180,83_1124642_17074019-82_.jpg,1,0,0,0,0,1,0,...,0,1,1,1,1,1,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,86_1215223,-1.357636,86_1215223_47014392-01_B.jpg,1,0,0,0,0,1,0,...,0,0,1,1,1,1,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
3,84_1167695,0.398726,84_1167695_27005150-70_B.jpg,1,0,0,0,0,1,0,...,0,0,1,1,1,1,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,82_1108473,-1.357636,82_1108473_87076320-01_B.jpg,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61361,83_1126644,1.136907,83_1126644_17042513-99_B.jpg,1,0,0,0,1,0,0,...,0,0,1,1,1,1,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
61362,84_1160958,-0.364910,84_1160958_27081129-40_B.jpg,1,0,0,0,1,0,0,...,0,1,1,0,1,0,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
61363,82_1103726,-1.332181,82_1103726_87054384-02_B.jpg,1,0,0,0,0,0,1,...,0,0,1,1,1,1,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
61364,82_1087881,1.009634,82_1087881_87091012-94_B.jpg,1,0,0,0,0,1,0,...,0,1,1,1,1,1,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [20]:
result.to_csv('../data/preprocessed_training.csv')

In [23]:
# Create a lookup table by mapping each list element to a key
df = pd.DataFrame(des_value_list, columns=['Value'])
df['Key'] = [f'key_{i+1}' for i in range(len(df))]

# Reorder columns if needed (Key, Value)
df = df[['Key', 'Value']]

# Save the DataFrame as a CSV
df.to_csv('../data/des_value_list.csv', index=False)

In [24]:
import json

# Specify the filename to save the JSON data
filename = '../data/attribute_indices_dict.json'

# Open the file in write mode and use json.dump to write the dictionary as JSON
with open(filename, 'w') as json_file:
    json.dump(attribute_indices_dict, json_file, indent=4)  # The indent adds formatting to the JSON file

print(f"Dictionary saved as '{filename}'")

Dictionary saved as '../data/attribute_indices_dict.json'
